# Исследование установаления параметров недвижимости в г. Санкт-Петербурге.

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import calendar
import datetime

In [2]:
try:
    data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv', sep='\t')

In [3]:
print('Информация о датафрейме:')
data.shape

Информация о датафрейме:


(23699, 22)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

In [5]:
data.head(30)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


In [6]:
data.columns

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'cityCenters_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition'],
      dtype='object')

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Судя на пятой строчке проектной работы по установлению параметров и проведению предобработки и исследовательского анализа данных с архивными объявлениями о продаже квартир в г. Санкт-Петербург: мы обнаруживаем, что вместо представления данных в столбцах: 'studio' (наличие квартиры-студии) и 'open_plan' (наличие свободной планировки) обнаруживание пропуск данных в виде многоточия ('...') и неполную картину над параметрами всех объявлений о продаже квартир в г. Санкт-Петербург<br>
<br>
Поэтому чтобы показать полностью все названия столбцов с соответствующими значениями воспользуемся методом: pd.set_option('display.max_columns', None) и чтобы не запутаться с порядковыми номерами индексов применим свойство упорядочивания методом .sort_index(), а для числовых значений из-за главного критерия недвижимости в г. Санкт-Петербург: цены на момент снятия с публикации, т.е. округление до двух цифр после запятой: чтобы привести все числовые данные до двух нулей нужно прописать следующую строку pd.options.display.float_format = '{:,.2f}'.format.<br>
<br>
Ещё в шестой и восьмой строчке данной проектной работы, обнаруживаем неэтическое для программистов название столбца: 'cityCenters_nearest', поэтому сразу меняем это название на 'city_сenters_nearest', не меняя расположения в таблице data.<br>    
</div>

In [7]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
data.sample(10).sort_index()

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
3257,15,"3,400,000.00",41.00,2018-09-21T00:00:00,1,NaN,18.00,15.70,3,NaN,False,False,11.60,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,62.00
9291,3,"3,100,000.00",57.00,2018-02-23T00:00:00,2,2.50,5.00,32.00,4,NaN,False,False,9.00,2.00,Гатчина,NaN,NaN,NaN,NaN,NaN,NaN,138.00
14113,2,"5,100,000.00",42.00,2018-06-02T00:00:00,1,2.70,16.00,19.00,4,NaN,False,False,10.20,NaN,Санкт-Петербург,"34,836.00","15,138.00",0.00,NaN,1.00,805.00,199.00
14993,1,"4,746,000.00",79.10,2018-10-29T00:00:00,2,2.80,9.00,43.20,4,False,False,False,14.40,NaN,Всеволожск,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15234,6,"4,630,000.00",43.60,2017-11-10T00:00:00,1,NaN,17.00,18.60,8,NaN,False,False,13.60,2.00,Санкт-Петербург,"37,434.00","8,460.00",1.00,441.00,2.00,427.00,35.00
15641,13,"3,786,000.00",35.00,2017-06-29T00:00:00,1,2.50,4.00,16.00,1,NaN,False,False,8.00,NaN,Сестрорецк,"55,893.00","32,823.00",0.00,NaN,1.00,468.00,642.00
17427,10,"3,900,000.00",32.40,2019-03-01T00:00:00,2,2.50,12.00,22.00,3,NaN,False,False,NaN,NaN,Санкт-Петербург,"18,230.00","21,392.00",0.00,NaN,0.00,NaN,NaN
17563,6,"24,500,000.00",167.50,2017-10-07T00:00:00,3,NaN,6.00,83.60,6,NaN,False,False,54.20,1.00,Санкт-Петербург,"24,126.00","2,291.00",2.00,305.00,3.00,329.00,72.00
23042,12,"11,500,000.00",85.00,2018-08-16T00:00:00,2,3.20,4.00,48.20,4,NaN,False,False,8.20,NaN,Санкт-Петербург,"21,711.00","1,826.00",2.00,628.00,1.00,78.00,NaN
23663,12,"6,400,000.00",88.00,2018-01-12T00:00:00,4,2.55,10.00,NaN,1,NaN,False,False,NaN,NaN,Санкт-Петербург,"33,642.00","13,943.00",0.00,NaN,2.00,844.00,59.00


In [9]:
data.rename(columns = {'cityCenters_nearest' : 'city_centers_nearest'}, inplace = True)

<div class="alert alert-info">
<b>Комментарий студента:</b>
И в конец обзора датафрейма data про аренды недвижимости в г. Санкт-Петербурге выведем характеристику значений всех столбцов числовых значений.<br>    
</div>

In [10]:
data.describe()

,total_images,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
count,"23,699.00","23,699.00","23,699.00","23,699.00","14,504.00","23,613.00","21,796.00","23,699.00","21,421.00","12,180.00","18,157.00","18,180.00","18,181.00","8,079.00","18,181.00","9,110.00","20,518.00"
mean,9.86,"6,541,548.77",60.35,2.07,2.77,10.67,34.46,5.89,10.57,1.15,"28,793.67","14,191.28",0.61,490.80,0.77,517.98,180.89
std,5.68,"10,887,013.27",35.65,1.08,1.26,6.60,22.03,4.89,5.91,1.07,"12,630.88","8,608.39",0.80,342.32,0.94,277.72,219.73
min,0.00,"12,190.00",12.00,0.00,1.00,1.00,2.00,1.00,1.30,0.00,0.00,181.00,0.00,1.00,0.00,13.00,1.00
25%,6.00,"3,400,000.00",40.00,1.00,2.52,5.00,18.60,2.00,7.00,0.00,"18,585.00","9,238.00",0.00,288.00,0.00,294.00,45.00
50%,9.00,"4,650,000.00",52.00,2.00,2.65,9.00,30.00,4.00,9.10,1.00,"26,726.00","13,098.50",0.00,455.00,1.00,502.00,95.00
75%,14.00,"6,800,000.00",69.90,3.00,2.80,16.00,42.30,8.00,12.00,2.00,"37,273.00","16,293.00",1.00,612.00,1.00,729.00,232.00
max,50.00,"763,000,000.00",900.00,19.00,100.00,60.00,409.70,33.00,112.00,5.00,"84,869.00","65,968.00",3.00,"3,190.00",3.00,"1,344.00","1,580.00"


 ### Шаг 2. Заполнение пропусков.

In [11]:
data.isna().sum()

total_images                0
last_price                  0
total_area                  0
first_day_exposition        0
rooms                       0
ceiling_height           9195
floors_total               86
living_area              1903
floor                       0
is_apartment            20924
studio                      0
open_plan                   0
kitchen_area             2278
balcony                 11519
locality_name              49
airports_nearest         5542
city_centers_nearest     5519
parks_around3000         5518
parks_nearest           15620
ponds_around3000         5518
ponds_nearest           14589
days_exposition          3181
dtype: int64

In [12]:
null_columns = []

for value in data.columns: 
    if data[value].count() != data.shape[0]:
        null_columns.append(value)
        
null_columns

['ceiling_height',
 'floors_total',
 'living_area',
 'is_apartment',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'city_centers_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

In [13]:
for item in null_columns:
    print(f"\n Строки датафрейма, где у столбца '{item}' обнаружены пропущенные значения:")
    display(data[data[item].isna()].head())


 Строки датафрейма, где у столбца 'ceiling_height' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11.00,18.60,1,NaN,False,False,11.00,2.00,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81.00
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5.00,34.30,4,NaN,False,False,8.30,0.00,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558.00
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14.00,NaN,9,NaN,False,False,NaN,0.00,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424.00
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12.00,14.40,5,NaN,False,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.00
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26.00,10.60,6,NaN,False,False,14.40,1.00,посёлок Парголово,"52,996.00","19,143.00",0.00,NaN,0.00,NaN,155.00



 Строки датафрейма, где у столбца 'floors_total' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
186,12,"11,640,000.00",65.20,2018-10-02T00:00:00,2,NaN,NaN,30.80,4,NaN,False,False,12.00,NaN,Санкт-Петербург,"39,197.00","12,373.00",1.00,123.00,0.00,NaN,49.00
237,4,"2,438,033.00",28.10,2016-11-23T00:00:00,1,NaN,NaN,20.75,1,NaN,False,False,NaN,0.00,Санкт-Петербург,"22,041.00","17,369.00",0.00,NaN,1.00,374.00,251.00
457,4,"9,788,348.00",70.80,2015-08-01T00:00:00,2,NaN,NaN,38.40,12,NaN,False,False,10.63,0.00,Санкт-Петербург,"37,364.00","8,322.00",2.00,309.00,2.00,706.00,727.00
671,4,"6,051,191.00",93.60,2017-04-06T00:00:00,3,NaN,NaN,47.10,8,NaN,False,False,16.80,0.00,Санкт-Петербург,"22,041.00","17,369.00",0.00,NaN,1.00,374.00,123.00
1757,5,"3,600,000.00",39.00,2017-04-22T00:00:00,1,NaN,NaN,NaN,9,NaN,False,False,NaN,0.00,Санкт-Петербург,"22,735.00","11,618.00",1.00,835.00,1.00,652.00,77.00



 Строки датафрейма, где у столбца 'living_area' обнаружены пропущенные значения:


...

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12.00,14.40,5,NaN,False,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.00
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27.00,15.43,26,NaN,False,False,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.00
12,10,"3,890,000.00",54.00,2016-06-30T00:00:00,2,NaN,5.00,30.00,5,NaN,False,False,9.00,0.00,Сертолово,NaN,NaN,NaN,NaN,NaN,NaN,90.00
22,20,"5,000,000.00",58.00,2017-04-24T00:00:00,2,2.75,25.00,30.00,15,NaN,False,False,11.00,2.00,деревня Кудрово,NaN,NaN,NaN,NaN,NaN,NaN,60.00
30,12,"2,200,000.00",32.80,2018-02-19T00:00:00,1,NaN,9.00,NaN,2,NaN,False,False,NaN,NaN,Коммунар,NaN,NaN,NaN,NaN,NaN,NaN,63.00



 Строки датафрейма, где у столбца 'ponds_nearest' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11.00,18.60,1,NaN,False,False,11.00,2.00,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81.00
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12.00,14.40,5,NaN,False,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.00
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26.00,10.60,6,NaN,False,False,14.40,1.00,посёлок Парголово,"52,996.00","19,143.00",0.00,NaN,0.00,NaN,155.00
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24.00,NaN,22,NaN,False,False,18.90,2.00,Санкт-Петербург,"23,982.00","11,634.00",0.00,NaN,0.00,NaN,NaN
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27.00,15.43,26,NaN,False,False,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.00



 Строки датафрейма, где у столбца 'days_exposition' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16.00,51.00,8,NaN,False,False,25.00,NaN,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,NaN
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24.00,NaN,22,NaN,False,False,18.90,2.00,Санкт-Петербург,"23,982.00","11,634.00",0.00,NaN,0.00,NaN,NaN
44,13,"5,350,000.00",40.00,2018-11-18T00:00:00,1,NaN,22.00,NaN,3,NaN,False,False,NaN,1.00,Санкт-Петербург,"30,471.00","11,603.00",1.00,620.00,1.00,"1,152.00",NaN
45,17,"5,200,000.00",50.60,2018-12-02T00:00:00,2,2.65,9.00,30.30,7,NaN,False,False,7.00,NaN,Санкт-Петербург,"30,011.00","12,872.00",0.00,NaN,0.00,NaN,NaN
46,17,"6,600,000.00",52.10,2019-01-31T00:00:00,2,2.60,24.00,29.70,9,NaN,False,False,8.30,2.00,Санкт-Петербург,"15,114.00","12,702.00",0.00,NaN,0.00,NaN,NaN


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Теперь нужно заполнить пропущенные значения, но, к счастью, среди некоторых данных столбцов есть такие данные, которые можно логически определить чисто из здравого смысла, реже с помощью условий из других значений столбцов.<br>  
<br> Ими относятся: 
    <ul> 'is_apartment' (наличие аппартаментов); </ul>
    <ul> 'balcony' (наличие балконов); </ul>
    <ul> 'locality_name' (название населённого пункта); </ul>
    <ul> 'days_exposition' (длительность размещения объявления (от публикации до снятия)); </ul><br>, а также: 
    <ul> 'floors_total' (количество этажей в доме); </ul> 
    <ul> 'parks_nearest' (расстояние до ближайшего парка); </ul>
    <ul> 'ponds_nearest' (расстояние до ближайшего пруда). </ul>          
    
<br>    
    Судя на третьей строчке проектной работы по установлению параметров и проведению предобработки и исследовательского анализа данных с архивными объявлениями о продаже квартир в г. Санкт-Петербург: мы обнаруживаем, что вместо представления данных в столбцах: 'studio' (наличие квартиры-студии) и 'open_plan'. <br>
<br>
     Также данные столбцов 'balcony', 'floors_total', а также 'days_exposition' приведём к типу данных 'int' (насчёт столбца 'days_exposition' - ни один SEO-специалист не будет ослеживать длительность с точностью до минуты, для удобства длительность размещения объявления будет измеряться в целых числах дней).<br>
</div>

In [14]:
data['is_apartment'] = data['is_apartment'].fillna('False')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16.00,51.00,8,False,False,False,25.00,NaN,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,NaN
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11.00,18.60,1,False,False,False,11.00,2.00,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81.00
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5.00,34.30,4,False,False,False,8.30,0.00,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558.00
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14.00,NaN,9,False,False,False,NaN,0.00,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424.00
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14.00,32.00,13,False,False,False,41.00,NaN,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121.00


In [15]:
data['balcony'] = (data['balcony'].fillna(0)).astype('int')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16.00,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,NaN
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11.00,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81.00
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5.00,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558.00
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14.00,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424.00
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14.00,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121.00


In [16]:
data_days_exposition_median = data['days_exposition'].median()
data_days_exposition_median

95.0

In [17]:
data['days_exposition'] = (data['days_exposition'].fillna(data_days_exposition_median)).astype('int')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16.00,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11.00,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5.00,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14.00,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14.00,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121


In [18]:
data_floors_total_median = data['floors_total'].median()
data_floors_total_median

9.0

In [19]:
data['floors_total'] = (data['floors_total'].fillna(data_floors_total_median)).astype('int')
display(data.head(50))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,NaN,0.00,NaN,81
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,NaN,0.00,NaN,155
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,NaN,0.00,NaN,95
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,NaN,0.00,NaN,289


In [20]:
data.loc[~(data['parks_around3000'] != data['parks_around3000']), 'parks_nearest'] = data.loc[:, 'parks_nearest'].fillna(0)

display(data.head(50)) 

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,0.00,0.00,NaN,81
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,0.00,0.00,NaN,155
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,0.00,0.00,NaN,95
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,0.00,0.00,NaN,289


In [21]:
data.loc[~(data['ponds_around3000'] != data['ponds_around3000']), 'ponds_nearest'] = (data.loc[:, 'ponds_nearest'].fillna(0.0)).astype('int')

display(data.head(50))  

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,0.00,0.00,0.00,81
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,0.00,0.00,0.00,155
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,0.00,0.00,0.00,95
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,0.00,0.00,0.00,289


### Шаг 3. Посчитайте и добавьте в таблицу.

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>Чтобы подсчитать цену квадратнного метра (столбец 'price_sqmetr') нужно разделить значение цены недвижимости (столбец 'last_price') на значение общей площади аппаратментов (столбец 'total_area').<br>
</div>

In [22]:
data['price_sqmetr'] = (data['last_price'] / data['total_area']).astype('int')
display(data.head(10))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,price_sqmetr
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95,120370
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,0.00,0.00,0.00,81,82920
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558,92785
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424,408176
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121,100000
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55,95065
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,0.00,0.00,0.00,155,99195
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,0.00,0.00,0.00,95,110544
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189,87454
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,0.00,0.00,0.00,289,88524


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>Для определения 
    
    
    (столбец 'price_sqmetr') нужно разделить значение цены недвижимости (столбец 'last_price') на значение общей площади аппаратментов (столбец 'total_area'). 
    
    
    
    
    Сейчас встаим в столбец 'airports_nearest' расстояние до ближайшего аэропорта в метрах и сделаем, с помощью вставки 
в первую строку проекта библиотеки: from geopy.distance import geodesic as gd (определение расстояния между двумя географическими координатами) и from geopy.geocoders import Nominatim as nm (определение широты и долготы координат на карте)
    
    
    
    
    
<br><br><b><i>Внимание!</i></b> Процесс опредения широты и долготы по данному населённому пункту происходит дольше обычного выполнения команды программ.<br>
</div>

In [23]:
def name_weekday(number_name):
    try:
        if(pd.DatetimeIndex(data['first_day_exposition']).weekday == 0):
            return 'понедельник'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 1):
            return 'вторник'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 2):
            return 'среда'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 3):
            return 'четверг'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 4):
            return 'пятница'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 5):
            return 'суббота'
        elif(pd.DatetimeIndex(data['first_day_exposition']).weekday == 6):
            return 'воскресенье'
    except:
        pass

In [24]:
data['weekday'] = pd.DatetimeIndex(data['first_day_exposition']).weekday

#name_weekday(pd.DatetimeIndex(data['first_day_exposition']).weekday.all())

#data['weekday'] = data['first_day_exposition'].apply(name_weekday)
data['number_month'] = pd.DatetimeIndex(data['first_day_exposition']).month
data['year'] = pd.DatetimeIndex(data['first_day_exposition']).year 

display(data.head(10))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,price_sqmetr,weekday,number_month,year
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95,120370,3,3,2019
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,0.00,0.00,0.00,81,82920,1,12,2018
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558,92785,3,8,2015
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424,408176,4,7,2015
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121,100000,1,6,2018
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55,95065,0,9,2018
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,0.00,0.00,0.00,155,99195,3,11,2017
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,0.00,0.00,0.00,95,110544,3,4,2019
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189,87454,2,5,2018
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,0.00,0.00,0.00,289,88524,6,2,2017


In [25]:
data['ratio_living_to_total'] = data['living_area'] / data['total_area']
data['ratio_kitchen_to_total'] = data['kitchen_area'] / data['total_area']

In [26]:
data.head(10)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,price_sqmetr,weekday,number_month,year,ratio_living_to_total,ratio_kitchen_to_total
0,20,"13,000,000.00",108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,False,False,25.00,0,Санкт-Петербург,"18,863.00","16,028.00",1.00,482.00,2.00,755.00,95,120370,3,3,2019,0.47,0.23
1,7,"3,350,000.00",40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,False,False,11.00,2,посёлок Шушары,"12,817.00","18,603.00",0.00,0.00,0.00,0.00,81,82920,1,12,2018,0.46,0.27
2,10,"5,196,000.00",56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,False,False,8.30,0,Санкт-Петербург,"21,741.00","13,933.00",1.00,90.00,2.00,574.00,558,92785,3,8,2015,0.61,0.15
3,0,"64,900,000.00",159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,False,False,NaN,0,Санкт-Петербург,"28,098.00","6,800.00",2.00,84.00,3.00,234.00,424,408176,4,7,2015,NaN,NaN
4,2,"10,000,000.00",100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,False,False,41.00,0,Санкт-Петербург,"31,856.00","8,098.00",2.00,112.00,1.00,48.00,121,100000,1,6,2018,0.32,0.41
5,10,"2,890,000.00",30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,False,False,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55,95065,0,9,2018,0.47,0.30
6,6,"3,700,000.00",37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,False,False,14.40,1,посёлок Парголово,"52,996.00","19,143.00",0.00,0.00,0.00,0.00,155,99195,3,11,2017,0.28,0.39
7,5,"7,915,000.00",71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,False,False,18.90,2,Санкт-Петербург,"23,982.00","11,634.00",0.00,0.00,0.00,0.00,95,110544,3,4,2019,NaN,0.26
8,20,"2,900,000.00",33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,False,False,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189,87454,2,5,2018,0.47,0.27
9,18,"5,400,000.00",61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,False,False,6.50,2,Санкт-Петербург,"50,898.00","15,008.00",0.00,0.00,0.00,0.00,289,88524,6,2,2017,0.71,0.11


### Шаг 3.1 Подсчёт данных и добавление в новые столбцы.

### 0.4 Шаг 4.1 Проведите исследовательский анализ данных.

## 1 Общий вывод: